#  Execute tasks via MPI on single or multiple batch nodes

AzureClusterlessHPC supports running single or multi-node tasks with distributed memory parallelism and Julia's MPI package. In combination with batch processing, AzureClusterlessHPC therefore allows users to execute multiple MPI tasks in parallel. Concurrent MPI tasks can be executed (a) in the same batch pool; (b) in separate batch pools beloning to the same batch account; (c) in separate batch pools belonging to separate batch accounts.

## Set up

To enable MPI jobs, set the following parameters in the `parameters.json` file:

- `MPI_RUN`: Set to "`1"` to enable execution of tasks as MPI jobs (default is `"0"`). If set to `"1"`, AzureClusterlessHPC will execute the user code on the batch node via `mpiexecjl`.

- `"_INTER_NODE_CONNECTION"`: Set to `"1"` if you want to run each MPI tasks across multiple nodes or set to `"0"` if you want to run each MPI task on a single node (e.g. for accessing multiple sockets on the same node).


- `"_NUM_NODES_PER_TASK"`: The number of nodes per task. Note that if you set this parameter > 1, `"_INTER_NODE_CONNECTION"` must be set to `"1"` (otherwise AzureClusterlessHPC defaults to 1 node per task).


- `"_NUM_PROCS_PER_NODE"`: Number of MPI ranks/processes per node. The total number of MPI ranks is given by `"_NUM_NODES_PER_TASK"` times `"_NUM_PROCS_PER_NODE"`. E.g., if you use two nodes per task and 4 processes per node, the total number of MPI ranks is 8.


- `"_OMP_NUM_THREADS"`: Number of OpenMP threads (if applicable; default is `"1"`).

In [1]:
# Install Julia packages required for this example
using Pkg; Pkg.add("MPI")

# Set path to credentials
ENV["CREDENTIALS"] = joinpath(pwd(), "../..", "credentials.json")

# Set path to batch parameters (pool id, VM types, etc.)
ENV["PARAMETERS"] = joinpath(pwd(), "parameters.json")

# Load package
using AzureClusterlessHPC
batch_clear();

Next, we start our batch pool. If `"_INTER_NODE_CONNECTION"` is set to `"1"`, AzureClusterlessHPC enables inter-node communication in the batch pool(s).

In [2]:
# Create pool
startup_script = "pool_startup_script.sh"
create_pool_and_resource_file(startup_script);

Created pool 1 of 2 in canadacentral with 2 nodes.
Created pool 2 of 2 in canadacentral with 2 nodes.


## MPI jobs with AzureClusterlessHPC

To execute MPI function, we need to load the MPI package with `@batchdef` and call `MPI.Init()`:

In [3]:
@batchdef using MPI
@batchdef MPI.Init();

Next, we define our function that will be executed by Azure Batch through AzureClusterlessHPC. With MPI, this function can be executed on either a single or multiple batch nodes.

In [4]:
# Parallel MPI function
@batchdef function hello_world()
    comm = MPI.COMM_WORLD
    print("Hello world, I am rank $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))\n")
    MPI.Barrier(comm)
    return "Goodbye"
end;

We can execute a single instance of this function using the `@batchexec` macro:

In [5]:
# Execute one MPI job
@batchexec hello_world();

We can also execute multiple version of the function in parallel using the `pmap` function. In this case, we execute two separate MPI jobs in parallel and collect their results:

In [6]:
# Execute multiple MPI jobs in parallel
bctrl = @batchexec pmap(i -> hello_world(), 1:2);

  1.617372 seconds (96.32 k allocations: 5.929 MiB, 6.65% compilation time)


In [7]:
# Collect output
out = fetch(bctrl)
print(out);

Monitoring tasks for 'Completed' state, timeout in 60 minutes ...Creating job [BatchJob_QWS4rqHk_1]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/application-cmd to blob container [azureclusterlesstemp]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/batch_runtime.jl to blob container [azureclusterlesstemp]...
Uploading file packages.dat to container [azureclusterlesstemp]...
Uploading file task_1.dat to container [azureclusterlesstemp]...
Creating job [BatchJob_BYJEmazC_1]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/application-cmd to blob container [azureclusterlesstemp]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/batch_runtime.jl to blob container [azureclusterlesstemp]...
Uploading file packages.dat to container [azureclusterlesstemp]...
Uploading file task_1.dat to container [azureclusterlesstemp]...
Creating job [BatchJob_BYJEmazC_2]...
Uploading file /home/pwitte/.ju

At the end, we clean up all consumed Azure resources:

In [8]:
destroy!(bctrl);

## Copyright

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License (MIT). See LICENSE in the repo root for license information.